In [242]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Initialize Chrome options
chrome_options = Options()

# Set up date formatting for today's date
today = datetime.today().strftime('%Y-%m-%d')


In [243]:
#LinkedIn Credentials
username=""
password=""

#Url of the Linkedin Page you want to scrape
page = 'https://www.linkedin.com/company/nike'

In [244]:
#access Webriver
browser = webdriver.Chrome()

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

In [245]:
#Enter login info:
elementID = browser.find_element(By.ID, "username")
elementID.send_keys(username)

elementID = browser.find_element(By.ID, "password")
elementID.send_keys(password)

elementID.submit()

In [246]:
#Go to webpage
post_page = page + '/posts'
post_page = post_page.replace('//posts','/posts')
browser.get(post_page)

In [247]:
if page[-1] == "/":
    company_name = page.split("/")[-2]
else:
    company_name = page.split("/")[-1]
    
company_name = company_name.replace('-',' ').title()
print(company_name)

Nike


In [316]:
import time

SCROLL_PAUSE_TIME = 1.5
MAX_SCROLLS = False

# JavaScript commands
SCROLL_COMMAND = "window.scrollTo(0, document.body.scrollHeight);"
GET_SCROLL_HEIGHT_COMMAND = "return document.body.scrollHeight"

# Initial scroll height
last_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
scrolls = 0
no_change_count = 0

while True:
    # Scroll down to bottom
    browser.execute_script(SCROLL_COMMAND)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
    
    # Increment no change count or reset it
    no_change_count = no_change_count + 1 if new_height == last_height else 0
    
    # Break loop if the scroll height has not changed for 3 cycles or reached the maximum scrolls
    if no_change_count >= 3 or (MAX_SCROLLS and scrolls >= MAX_SCROLLS):
        break
    
    last_height = new_height
    scrolls += 1


In [317]:
company_page = browser.page_source
linkedin_soup = bs(company_page.encode("utf-8"), "html")

with open(f"{company_name}_soup.txt", "w+") as t:
    t.write(linkedin_soup.prettify())
#print(linkedin_soup.prettify())

containers = linkedin_soup.find_all("div",{"class":"feed-shared-update-v2"})
containers = [container for container in containers if 'activity' in container.get('data-urn', '')]
print(len(containers))

containers_text = "\n\n".join([c.prettify() for c in containers])
with open(f"{company_name}_soup_containers.txt", "w+") as t:
    t.write(containers_text)

38


In [318]:
def get_actual_date(date):
    today = datetime.today().strftime('%Y-%m-%d')
    current_year = datetime.today().strftime('%Y')
    
    def get_past_date(days=0, weeks=0, months=0, years=0):
        date_format = '%Y-%m-%d'
        dtObj = datetime.strptime(today, date_format)
        past_date = dtObj - relativedelta(days=days, weeks=weeks, months=months, years=years)
        past_date_str = past_date.strftime(date_format)
        return past_date_str

    past_date = date
    
    if 'hour' in date:
        past_date = today
    elif 'day' in date:
        date.split(" ")[0]
        past_date = get_past_date(days=int(date.split(" ")[0]))
    elif 'week' in date:
        past_date = get_past_date(weeks=int(date.split(" ")[0]))
    elif 'month' in date:
        past_date = get_past_date(months=int(date.split(" ")[0]))
    elif 'year' in date:
        past_date = get_past_date(months=int(date.split(" ")[0]))
    else:
        split_date = date.split("-")
        if len(split_date) == 2:
            past_month = split_date[0]
            past_day =  split_date[1]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{current_year}-{past_month}-{past_day}"
        elif len(split_date) == 3:
            past_month = split_date[0]
            past_day =  split_date[1]
            past_year = split_date[2]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{past_year}-{past_month}-{past_day}"

    return past_date



def convert_abbreviated_to_number(s):
    if 'K' in s:
        return int(float(s.replace('K', '')) * 1000)
    elif 'M' in s:
        return int(float(s.replace('M', '')) * 1000000)
    else:
        return int(s)

In [319]:
import re

# Define a data structure to hold all the post information
posts_data = []

def get_text(container, selector, attributes):
    try:
        element = container.find(selector, attributes)
        if element:
            return element.text.strip()
    except Exception as e:
        print(e)
    return ""

# Function to extract media information
def get_media_info(container):
    media_info = [("div", {"class": "update-components-video"}, "Video"),
                  ("div", {"class": "update-components-linkedin-video"}, "Video"),
                  ("div", {"class": "update-components-image"}, "Image"),
                  ("article", {"class": "update-components-article"}, "Article"),
                  ("div", {"class": "feed-shared-external-video__meta"}, "Youtube Video"),
                  ("div", {"class": "feed-shared-mini-update-v2 feed-shared-update-v2__update-content-wrapper artdeco-card"}, "Shared Post"),
                  ("div", {"class": "feed-shared-poll ember-view"}, "Other: Poll, Shared Post, etc")]
    
    for selector, attrs, media_type in media_info:
        element = container.find(selector, attrs)
        if element:
            link = element.find('a', href=True)
            return link['href'] if link else "None", media_type
    return "None", "Unknown"


# Main loop to process each container
for container in containers:
    post_text = get_text(container, "div", {"class": "feed-shared-update-v2__description-wrapper"})
    media_link, media_type = get_media_info(container)
    post_date = get_text(container, "div", {"class": "ml4 mt2 text-body-xsmall t-black--light"})
    post_date = get_actual_date(post_date)
    
    # Reactions (likes)
    reactions_element = container.find_all(lambda tag: tag.name == 'button' and 'aria-label' in tag.attrs and 'reaction' in tag['aria-label'].lower())
    reactions_idx = 1 if len(reactions_element) > 1 else 0
    post_reactions = reactions_element[reactions_idx].text.strip() if reactions_element and reactions_element[reactions_idx].text.strip() != '' else 0

    # Comments
    comment_element = container.find_all(lambda tag: tag.name == 'button' and 'aria-label' in tag.attrs and 'comment' in tag['aria-label'].lower())
    comment_idx = 1 if len(comment_element) > 1 else 0
    post_comments = comment_element[comment_idx].text.strip() if comment_element and comment_element[comment_idx].text.strip() != '' else 0

    # Shares
    shares_element = container.find_all(lambda tag: tag.name == 'button' and 'aria-label' in tag.attrs and 'repost' in tag['aria-label'].lower())
    shares_idx = 1 if len(shares_element) > 1 else 0
    post_shares = shares_element[shares_idx].text.strip() if shares_element and shares_element[shares_idx].text.strip() != '' else 0
        
        
    # Append the collected data to the posts_data list
    posts_data.append({
        "Page": company_name,
        "Date": post_date,
        "Post Text": post_text,
        "Media Type": media_type,
        "Likes": post_reactions,
        "Comments": post_comments,
        "Shares":post_shares,
        "Likes Numeric": convert_abbreviated_to_number(post_reactions),
        "Media Link": media_link
    })



In [320]:
try:
    df = pd.DataFrame(posts_data)
except:
    for item in list(data.keys()):
        print(item)
        print(len(data[item]))
        
for col in df.columns:
    try:
        df[col] = df[col].astype(int)
    except:
        pass
    
df.sort_values(by="Likes Numeric", inplace=True, ascending=False)
df

,Page,Date,Post Text,Media Type,Likes,Comments,Shares,Likes Numeric,Media Link
1,Nike,2023-11-06,Offering employees the best workspaces and wor...,Image,10K,109,245,10000,None
19,Nike,2023-06-27,"Dreaming up new expressions of sport, new ways...",Video,5.1K,72,320,5100,None
10,Nike,2023-06-27,"Breakthrough Nike products, cutting-edge innov...",Article,4.9K,85,60,4900,https://www.linkedin.com/newsletters/swoosh-st...
20,Nike,2022-12-27,"No secret that at Nike, we love shoes — and we...",Video,4K,37,177,4000,None
34,Nike,2023-10-27,Last Friday marked the opening of Jordan Brand...,Image,3.8K,64,197,3800,None
4,Nike,2023-10-27,Twenty years of dominating performances. Twent...,Image,3.6K,55,122,3600,None
9,Nike,2023-07-27,How does Nike define holistic fitness? The foc...,Video,3.3K,45,208,3300,None
37,Nike,2023-10-27,The world just got a first look at the Nike Ph...,Image,3.3K,90,137,3300,None
6,Nike,2023-06-27,"Juneteenth, also known as Freedom Day, commemo...",Image,2.9K,26,79,2900,None
2,Nike,2023-10-27,"Do you hear it? The dribble of the ball, the s...",Article,2.8K,48,70,2800,https://www.linkedin.com/newsletters/swoosh-st...


In [321]:
df.to_csv("{}_posts.csv".format(company_name), encoding='utf-8', index=False)
# writer = pd.ExcelWriter("{}_linkedin_posts.xlsx".format(company_name), engine='xlsxwriter')
# df.to_excel(writer, index = False)
# writer.save()